<a href="https://colab.research.google.com/github/zulfiqaralimir/AI-Powered-SocialMediaPostOptimizer_LLM/blob/main/Multi_AI_Agents_Chatbots_With_External_Tools_With_LangGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing Liberaries

In [ ]:
!pip install langgraph langsmith langchain langchain_groq langchain_community

# For State Management

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict

In [ ]:
!pip install arxiv wikipedia

# Working with External Tools
### **Arxiv API Integration**: The wrapper effectively leverages the Arxiv API to perform efficient and accurate searches for scientific papers.

### Researchers can use this wrapper to quickly find and explore relevant scientific literature on their areas of interest, saving time and effort.

In [ ]:
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun

In [ ]:
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=300)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [ ]:
# Create a Wikipedia API wrapper
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=300)
# Create a Wikipedia query tool
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [ ]:
# Invoke the tool with the query
result = wiki_tool.invoke("What is Generative AI?")

# Print the result
print(result)